# E4S Container Environment Debug

This notebook helps identify the correct paths within the Singularity container environment to access the VOC dataset.

In [ ]:
import os
import subprocess

print("=== CONTAINER ENVIRONMENT ANALYSIS ===")
print(f"Current working directory: {os.getcwd()}")
print(f"User: {os.environ.get('USER', 'unknown')}")
print(f"Home directory: {os.environ.get('HOME', 'unknown')}")
print(f"Path: {os.environ.get('PATH', 'unknown')[:200]}...")

In [ ]:
# Check what's mounted and available
print("=== ROOT DIRECTORY CONTENTS ===")
try:
    root_contents = os.listdir('/')
    for item in sorted(root_contents):
        item_path = f'/{item}'
        if os.path.isdir(item_path):
            print(f"📁 {item}/")
        else:
            print(f"📄 {item}")
except Exception as e:
    print(f"Error listing root: {e}")

In [ ]:
# Look for possible SharedContent and data paths
print("=== CHECKING POSSIBLE DATA PATHS ===")
search_paths = [
    '/SharedContent',
    '/home/SharedContent', 
    f'{os.environ.get("HOME", "/home")}/SharedContent',
    '/opt/hccs_shared',
    '/mnt',
    '/data',
    '/shared',
    '/workspace'
]

for path in search_paths:
    if os.path.exists(path):
        print(f"✅ EXISTS: {path}")
        try:
            contents = os.listdir(path)
            print(f"   Contents: {contents[:10]}")
            if len(contents) > 10:
                print(f"   ... and {len(contents)-10} more items")
        except Exception as e:
            print(f"   Error reading: {e}")
    else:
        print(f"❌ NOT FOUND: {path}")

In [ ]:
# Search for VOC dataset recursively
print("=== SEARCHING FOR VOC DATASET ===")

def find_voc_dataset(start_path, max_depth=2, current_depth=0):
    """Recursively search for VOC dataset"""
    if current_depth > max_depth:
        return []
    
    found_paths = []
    try:
        for item in os.listdir(start_path):
            if any(keyword in item.lower() for keyword in ['voc', 'pascal', '2012']):
                full_path = os.path.join(start_path, item)
                found_paths.append(full_path)
            
            # Recurse into subdirectories
            if current_depth < max_depth:
                item_path = os.path.join(start_path, item)
                if os.path.isdir(item_path) and not os.path.islink(item_path):
                    try:
                        found_paths.extend(find_voc_dataset(item_path, max_depth, current_depth + 1))
                    except (PermissionError, OSError):
                        pass
    except (PermissionError, OSError):
        pass
    
    return found_paths

# Search in key locations
search_locations = ['/', os.environ.get('HOME', '/home'), '/opt', '/mnt', '/data']
for location in search_locations:
    if os.path.exists(location):
        print(f"Searching in {location}...")
        voc_paths = find_voc_dataset(location)
        for voc_path in voc_paths:
            print(f"  🎯 FOUND: {voc_path}")
            # Check if it has the expected structure
            annotations_path = os.path.join(voc_path, 'VOC2012_train_val', 'Annotations')
            if os.path.exists(annotations_path):
                print(f"     ✅ Has Annotations folder!")
            else:
                # Check other possible structures
                for subdir in os.listdir(voc_path) if os.path.isdir(voc_path) else []:
                    sub_annotations = os.path.join(voc_path, subdir, 'Annotations')
                    if os.path.exists(sub_annotations):
                        print(f"     ✅ Found Annotations in {subdir}!")

# E4S Container Environment Debug

This notebook helps identify the correct paths within the Singularity container environment.

In [ ]:
import os
import subprocess

print("=== CONTAINER ENVIRONMENT ANALYSIS ===")
print(f"Current working directory: {os.getcwd()}")
print(f"User: {os.environ.get('USER', 'unknown')}")
print(f"Home directory: {os.environ.get('HOME', 'unknown')}")
print(f"Path: {os.environ.get('PATH', 'unknown')[:200]}...")

In [ ]:
# Check what's mounted and available
print("=== ROOT DIRECTORY CONTENTS ===")
try:
    root_contents = os.listdir('/')
    for item in sorted(root_contents):
        item_path = f'/{item}'
        if os.path.isdir(item_path):
            print(f"📁 {item}/")
        else:
            print(f"📄 {item}")
except Exception as e:
    print(f"Error listing root: {e}")

In [ ]:
# Check home directory
home_dir = os.environ.get('HOME', '/home')
print(f"=== HOME DIRECTORY CONTENTS ({home_dir}) ===")
try:
    home_contents = os.listdir(home_dir)
    for item in sorted(home_contents):
        item_path = os.path.join(home_dir, item)
        if os.path.islink(item_path):
            target = os.readlink(item_path)
            print(f"🔗 {item} -> {target}")
        elif os.path.isdir(item_path):
            print(f"📁 {item}/")
        else:
            print(f"📄 {item}")
except Exception as e:
    print(f"Error listing home: {e}")

In [ ]:
# Check if SharedContent exists and what's in it
shared_paths = [
    '/SharedContent',
    '/home/SharedContent', 
    f'{home_dir}/SharedContent',
    '/opt/hccs_shared',
    '/mnt',
    '/data'
]

print("=== CHECKING POSSIBLE SHARED CONTENT PATHS ===")
for path in shared_paths:
    if os.path.exists(path):
        print(f"✅ EXISTS: {path}")
        try:
            contents = os.listdir(path)
            print(f"   Contents: {contents[:10]}")  # Show first 10 items
            if len(contents) > 10:
                print(f"   ... and {len(contents)-10} more items")
        except Exception as e:
            print(f"   Error reading: {e}")
    else:
        print(f"❌ NOT FOUND: {path}")

In [ ]:
# Look for VOC dataset specifically
print("=== SEARCHING FOR VOC DATASET ===")
search_paths = [
    '/',
    home_dir,
    '/opt',
    '/mnt',
    '/data',
    '/shared',
    '/workspace'
]

def find_voc_dataset(start_path, max_depth=3, current_depth=0):
    """Recursively search for VOC dataset"""
    if current_depth > max_depth:
        return []
    
    found_paths = []
    try:
        for item in os.listdir(start_path):
            if 'voc' in item.lower() or 'pascal' in item.lower():
                full_path = os.path.join(start_path, item)
                found_paths.append(full_path)
            
            # Recurse into subdirectories
            if current_depth < max_depth:
                item_path = os.path.join(start_path, item)
                if os.path.isdir(item_path) and not os.path.islink(item_path):
                    found_paths.extend(find_voc_dataset(item_path, max_depth, current_depth + 1))
    except (PermissionError, OSError):
        pass
    
    return found_paths

for search_path in search_paths:
    if os.path.exists(search_path):
        print(f"Searching in {search_path}...")
        voc_paths = find_voc_dataset(search_path)
        for voc_path in voc_paths:
            print(f"  🎯 FOUND: {voc_path}")

In [ ]:
# Check mount points in container
print("=== MOUNT INFORMATION ===")
try:
    result = subprocess.run(['mount'], capture_output=True, text=True)
    mount_lines = result.stdout.split('\n')
    for line in mount_lines:
        if any(keyword in line.lower() for keyword in ['bind', 'shared', 'opt', 'hccs', 'data']):
            print(f"📌 {line}")
except Exception as e:
    print(f"Could not get mount info: {e}")

In [ ]:
# Check environment variables for hints
print("=== RELEVANT ENVIRONMENT VARIABLES ===")
env_keywords = ['SHARED', 'DATA', 'MOUNT', 'BIND', 'SINGULARITY', 'E4S', 'VOC', 'DATASET']
for key, value in os.environ.items():
    if any(keyword in key.upper() for keyword in env_keywords):
        print(f"🔑 {key}={value}")